# Outline

1. Identify key independent systems to simulate
2. Build separate simulations and visualizations for each one
3. Build interfaces between systems
4. Make combined metrics and controls over the whole sim
5. Notice emergent / interesting behavior, use as a guide for where to add detail

Repeat as necessary, ideally quickly

# Potential Systems

## Initial Systems

* Weather/geology (+lighting) (+zone)
* Ecosystem (food, animals, yadda yadda)
* Infrastructure
* Economic
* Social

## Potential Future Systems

* Energy Grid
* Regional Stuff
* Culture Groups  / [e|i]mmigration
* Memetics

# Potential Later Variables 



# Resources

## Videos
  + [What is Solarpunk?](https://www.youtube.com/watch?v=hHI61GHNGJM) (and [Andrewism](https://www.youtube.com/c/Andrewism))
  + [Places to build a home or village](https://www.youtube.com/watch?v=0Trz6NvsUQQ)
  + [Permaculture Design by Sectors](https://www.youtube.com/watch?v=233GgYhtoGs)
  + [Recycling Water](https://www.youtube.com/watch?v=f-sRcVkZ9yg)
  + [Andrew Millision](https://www.youtube.com/c/amillison)
  + [Simulating the Evolution of Predation](https://www.youtube.com/watch?v=rPkMoFJNcLA)

# System Design

## Ecosystem
Grid-based (diffusion) many-species [Lotka-Volterra](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations)
...or maybe no grid for now

In [1]:
import numpy as np

### Prey formula:

$x$ is the quantity of prey, $y$ is the quantity of predators.

$\alpha$ is the birthrate of the prey, $\beta$ is the rate at which predators eat the prey.


$$\frac{dx}{dt} = \alpha x - \beta x y$$


### Predators formula:

$\delta$ is the rate at which predators die naturally, $\gamma$ is the birthrate of the predators.

$$\frac{dy}{dt} = \delta x y - \gamma y$$


### Limitations

* Assumes prey has unlimited food

In [2]:
from dataclasses import dataclass


@dataclass
class LVEcosystemModel:
    """Lotka-Volterra predator/prey model. (https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations)"""

    alpha: float = 1.0
    """Birthrate of prey (x)."""
    beta: float = 1.0
    """Rate at which predators (y) eat prey (x)."""
    gamma: float = 1.0
    """Birthrate of predators (y)."""
    delta: float = 1.0
    """Rate at which predators die naturally."""

    x0: int = 10
    """Initial prey population."""
    y0: int = 2
    """Initial predators population."""

In [ ]:
# def LotkaVolterra(X, model=LVEcosystemModel()):
#     return np.array([
#         model.alpha * X[0] - model.beta * X[0] * X[1],
#         model.delta * X[0] * X[1] - model.gamma * X[1]
#     ])

# def LVJacobian(x, y, model=LVEcosystemModel()):
#     """Takes the jacobian matrix of a LotkaVolterra model."""
#     return np.matrix([[model.alpha - model.beta * y, -model.beta * x     ],
#                       [model.delta * y,               model.delta * x - y]])



Jacobian matrix of a function is the matrix of all first order partial derivatives, in this case:

$$
\begin{bmatrix}
\alpha - \beta y & -\beta x\\
\delta y & \delta x - y
\end{bmatrix}
$$

In [27]:
from scipy.integrate import odeint

model = LVEcosystemModel()

solution = odeint(
    LotkaVolterra,
    np.array([
       model.x0, 
       model.y0, 
    ]),
    np.linspace(0, 10, 500),
)

In [33]:
def f(x):
    return x

interact(f, x=10)

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.f(x)>

In [78]:
%matplotlib inline
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display

from collections.abc import Callable


def ParameterizedLV(model: LVEcosystemModel) -> Callable:
    """Function factory! Make an LV function that scipy odeint can call."""
    
    def LotkaVolterra2(X, t):
        return np.array([
            model.alpha * X[0] - model.beta * X[0] * X[1],
            model.delta * X[0] * X[1] - model.gamma * X[1]
        ])
    
    return LotkaVolterra2


def simulate(prey_birth: float, pred_eat_prey: float, pred_birth: float, pred_die: float, prey_initial: int, pred_initial: int, time_length: int, high_res: bool):
    model = LVEcosystemModel(
        alpha=prey_birth,
        beta=pred_eat_prey,
        gamma=pred_birth,
        delta=pred_die,
        x0=prey_initial,
        y0=pred_initial
    )
    
    if high_res:
        num_points = 1000
    else:
        num_points = 100
        
    timespace = np.linspace(0, time_length, num_points)
    
    solution = odeint(
        ParameterizedLV(model),
        np.array([
           model.x0, 
           model.y0, 
        ]),
        timespace,
    )
    
    fig, ax = plt.subplots(1, 2, figsize=(15,10))
    lines = ax[0].plot(solution)
    ax[0].set_title("Populations")
    ax[0].set_xlabel("Time")
    ax[0].set_ylabel("Pop")
    ax[0].legend(lines, ["Prey", "Pred"])

    ax[1].plot(solution[:, 0], solution[:, 1])
    ax[1].scatter(solution[0, 0], solution[0,1])
    ax[1].set_title("Phase")
    ax[1].set_xlabel("Prey")
    ax[1].set_ylabel("Pred")

    plt.show(fig)
    plt.close(fig)
    
    
display(interact(
    simulate,
    prey_birth=widgets.FloatSlider(min=0.0, max=10.0, step=.2, value=1.0),
    pred_eat_prey=widgets.FloatSlider(min=0.0, max=10.0, step=.2, value=1.0),
    pred_birth=widgets.FloatSlider(min=0.0, max=10.0, step=.2, value=1.0), 
    pred_die=widgets.FloatSlider(min=0.0, max=10.0, step=.2, value=1.0),
    prey_initial=widgets.IntSlider(min=1, max=100, step=1, value=10), 
    pred_initial=widgets.IntSlider(min=1, max=100, step=1, value=10),
    time_length=10,
    high_res=False
))

interactive(children=(FloatSlider(value=1.0, description='prey_birth', max=10.0, step=0.2), FloatSlider(value=…

<function __main__.simulate(prey_birth: float, pred_eat_prey: float, pred_birth: float, pred_die: float, prey_initial: int, pred_initial: int, time_length: int, high_res: bool)>

# Notes

Game design - if indicators are goals, what obstacles (do we put) in the way?

Multiple villages to see trade/economic effects etc?

Maybe to start with they exist in a vacuum (external infrastructure etc later)